# [User-defined Python functions](https://docs.pola.rs/user-guide/expressions/user-defined-python-functions/)

There a re two ways to do this:

- map_elements: Call a function separately on each value in the Series.
- map_batches: Always passes the full Series to the function.

## Processing individual values with `map_elements()`

In [1]:
import polars as pl
df = pl.DataFrame(
    {
        "keys": ["a", "a", "b", "b"],
        "values": [10, 7, 1, 23],
    }
)
print(df)

shape: (4, 2)
┌──────┬────────┐
│ keys ┆ values │
│ ---  ┆ ---    │
│ str  ┆ i64    │
╞══════╪════════╡
│ a    ┆ 10     │
│ a    ┆ 7      │
│ b    ┆ 1      │
│ b    ┆ 23     │
└──────┴────────┘


In [4]:
import math

def my_log(value):
    return math.log(value)

df.select(pl.col("values").map_elements(my_log, return_dtype=pl.Float64))

/tmp/ipykernel_20044/150826428.py:6: PolarsInefficientMapWarning: 
Expr.map_elements is significantly slower than the native expressions API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - pl.col("values").map_elements(my_log)
with this one instead:
  + pl.col("values").log()

  df.select(pl.col("values").map_elements(my_log, return_dtype=pl.Float64))


values
f64
2.302585
1.94591
0.0
3.135494


## Processing a whole Series with `map_batches()`

In [7]:
def diff_from_mean(series):
    total = 0
    for value in series:
        total += value
    mean = total / len(series)
    return pl.Series([value - mean for value in series])

# apply fucntion to full series
out = df.select(pl.col("values").map_batches(diff_from_mean))
print("== select() with UDF ==")
print(out)

# apply fucntion per group
out = df.group_by("keys").agg(pl.col("values").map_batches(diff_from_mean))
print("== group_by() with UDF ==")
print(out)

== select() with UDF ==
shape: (4, 1)
┌────────┐
│ values │
│ ---    │
│ f64    │
╞════════╡
│ -0.25  │
│ -3.25  │
│ -9.25  │
│ 12.75  │
└────────┘
== group_by() with UDF ==
shape: (2, 2)
┌──────┬───────────────┐
│ keys ┆ values        │
│ ---  ┆ ---           │
│ str  ┆ list[f64]     │
╞══════╪═══════════════╡
│ b    ┆ [-11.0, 11.0] │
│ a    ┆ [1.5, -1.5]   │
└──────┴───────────────┘


## Fast operations with user-defined functions